In [1]:
import os 
import numpy as np
from PIL import Image
import torch
import cv2
from tqdm import tqdm

import random
import matplotlib.pyplot as plt

import math

In [2]:
from totalface.model_zoo.model_common import load_openvino
from totalface.model_zoo.get_models import get_detection_model
from totalface.face.get_result import get_detection
from totalface.data import read_image
from totalface.utils.util_warp import face_align
from totalface.utils.util_common import draw_result,draw_result_land,draw_result_sim

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/_vendor/packaging/version.py:111: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pycuda/compyte/dtypes.py:120: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  reg.get_or_register_dtype("bool", np.bool)


In [3]:
from totalface.data.constant import LMARK_REF_ARC

def get_align(image,land5,LMARK_REF_ARC):
    aligned = face_align(image,LMARK_REF_ARC,land5,112)
    return aligned

In [4]:
# Load model

detection_name = "scrfd"

detection_path = ["scrfd_10g_bnkps_quantINT8.xml",
                 "scrfd_10g_bnkps_quantINT8.bin"]

# Set parameters
detection_thresh = 0.5
detection_height_min=0 

# if load multiple trt, load_multi=True, default False
detection_model = get_detection_model(detection_name,detection_path,load_multi=False)

scrfd openvino loaded


In [5]:
base = "rmfd_mfr2/mfr2_2/"
save_base = "rmfd_mfr2/mfr2_2_scrfd_aligned/"

if not os.path.exists(save_base):
    os.mkdir(save_base)

In [6]:
for idname in tqdm(os.listdir(base)):
    if ".DS" in idname: continue
    save_sub = os.path.join(save_base,idname)
    if not os.path.exists(save_sub):
        os.mkdir(save_sub)
    
    for img_name in os.listdir(os.path.join(base,idname)):
        if ".DS" in img_name: continue

        img_path = os.path.join(base,idname,img_name)
        
        img = read_image(img_path)
        faces = get_detection(detection_name,detection_model,img,thresh=detection_thresh,height_min=detection_height_min,input_size=(640,640))
        
        if len(faces)<1 or len(faces)>1:
            print("faces:",len(faces))
            continue
            
        face = faces[0]
        aligned_face = get_align(img,face.land5,LMARK_REF_ARC)
        if idname=='ArifAlvi':
            if 'unmask' in img_name:
                img_name = img_name.replace("unmask","mask")
            else:
                img_name = img_name.replace("mask","unmask")
                
        save_path = os.path.join(save_sub,img_name)
        cv2.imwrite(save_path,cv2.cvtColor(aligned_face,cv2.COLOR_RGB2BGR))
        

  6%|▌         | 3/49 [00:00<00:06,  7.33it/s]

faces: 2


 67%|██████▋   | 33/49 [00:04<00:01,  9.86it/s]

faces: 0


100%|██████████| 49/49 [00:11<00:00,  4.28it/s]
